In [1]:
from threading import Thread
from math import pi

import rclpy
from rclpy.callback_groups import ReentrantCallbackGroup
from rclpy.node import Node

from pymoveit2 import MoveIt2
from pymoveit2.robots import ur

from geometry_msgs.msg import Pose, Quaternion, Point
import rclpy.wait_for_message
from ur_msgs.srv import SetIO
from ur_commander.srv import VisualizePoses

rclpy.init()

# Create node for this example
node = Node("notebook_example")
callback_group = ReentrantCallbackGroup()


def display_poses(poses: list[Pose], frame_id: str = "base_link") -> None:
    client = node.create_client(VisualizePoses, "/visualize_poses")
    while not client.wait_for_service(timeout_sec=1.0):
        node.get_logger().info("service not available, waiting again...")
    client.call(VisualizePoses.Request(poses=poses, frame_id=frame_id))


moveit2 = MoveIt2(
    node=node,
    joint_names=ur.joint_names(),
    base_link_name=ur.base_link_name(),
    end_effector_name=ur.end_effector_name(),
    group_name=ur.MOVE_GROUP_ARM,
    callback_group=callback_group,
)

# Spin the node in background thread(s) and wait a bit for initialization
executor = rclpy.executors.MultiThreadedExecutor(2)
executor.add_node(node)
executor_thread = Thread(target=executor.spin, daemon=True, args=())
executor_thread.start()
node.create_rate(1.0).sleep()

# Scale down velocity and acceleration of joints (percentage of maximum)
moveit2.max_velocity = 0.1
moveit2.max_acceleration = 0.1
synchronous = True
cancel_after_secs = 0.0
cartesian = False
cartesian_max_step = 0.0025
cartesian_fraction_threshold = 0.0
cartesian_jump_threshold = 0.0
cartesian_avoid_collisions = False
moveit2.planner_id = "PTP"

# Add collision objects
moveit2.add_collision_box(
    id="table",
    size=[2.0, 1.0, 0.05],
    position=[0.0, 0.0, -0.025],
    quat_xyzw=[0.0, 0.0, 0.0, 0.0],
)

**Define Joint Position**   

Forward kinematics, a trajectory will be planned

In [2]:
home_position = [0.0, -pi / 2, pi / 2, 0.0, 0.0, 0.0]
node.get_logger().info("Moving to home position")
traj = moveit2.move_to_configuration(home_position)
if traj is None:
    node.get_logger().error("Failed to move to home position")

[INFO] [1732789810.603101304] [notebook_example]: Moving to home position


In [3]:
moveit2.execute(traj)
success = moveit2.wait_until_executed()
if not success:
    node.get_logger().error("Failed to execute trajectory")
else:
    node.get_logger().info("Moved to home position")

[INFO] [1732789821.803532923] [notebook_example]: Moved to home position


**Define a Target Pose**    

Display the pose

In [4]:
target = Pose(
    position=Point(x=0.6, y=0.0, z=0.5),
    orientation=Quaternion(x=0.0, y=0.7071067811865475, z=0.0, w=0.7071067811865476),
)

display_poses([target])

Plan a trajectory

In [5]:
position = [target.position.x, target.position.y, target.position.z]
quat_xyzw = [target.orientation.x, target.orientation.y, target.orientation.z, target.orientation.w]


node.get_logger().info(f"Moving to {{position: {list(position)}, quat_xyzw: {list(quat_xyzw)}}}")
traj = moveit2.move_to_pose(
    position=position,
    quat_xyzw=quat_xyzw,
    cartesian=False,
    cartesian_max_step=cartesian_max_step,
    cartesian_fraction_threshold=cartesian_fraction_threshold,
)

[INFO] [1732786259.874722188] [notebook_example]: Moving to {position: [0.6, 0.0, 0.5], quat_xyzw: [0.0, 0.7071067811865475, 0.0, 0.7071067811865476]}


Execute the trajectory

In [6]:
moveit2.execute(traj)
success = moveit2.wait_until_executed()
if not success:
    node.get_logger().error("Failed to execute trajectory")

In [4]:
target0 = Pose(
    position=Point(x=0.6, y=-0.3, z=0.5),
    orientation=Quaternion(x=0.0, y=0.7071067811865475, z=0.0, w=0.7071067811865476),
)
target1 = Pose(
    position=Point(x=0.6, y=0.3, z=0.5),
    orientation=Quaternion(x=0.5, y=0.5, z=0.5, w=0.5),
)

display_poses([target0, target1])

In [12]:
targets = [target0, target1]
for i, target_pose in enumerate(targets):

    position = [target_pose.position.x, target_pose.position.y, target_pose.position.z]
    quat_xyzw = [
        target_pose.orientation.x,
        target_pose.orientation.y,
        target_pose.orientation.z,
        target_pose.orientation.w,
    ]

    node.get_logger().info(f"Moving to target {i} {{position: {list(position)}, quat_xyzw: {list(quat_xyzw)}}}")
    traj = moveit2.move_to_pose(
        position=position,
        quat_xyzw=quat_xyzw,
        cartesian=cartesian,
        cartesian_max_step=cartesian_max_step,
        cartesian_fraction_threshold=cartesian_fraction_threshold,
    )
    if traj is not None:
        node.get_logger().info(f"Move to target {i}")
        moveit2.execute(traj)
        success = moveit2.wait_until_executed()

[INFO] [1732786447.653508915] [notebook_example]: Moving to target 0 {position: [0.6, -0.3, 0.5], quat_xyzw: [0.0, 0.7071067811865475, 0.0, 0.7071067811865476]}
[INFO] [1732786447.787047572] [notebook_example]: Move to target 0
[INFO] [1732786455.779066663] [notebook_example]: Moving to target 1 {position: [0.6, 0.3, 0.5], quat_xyzw: [0.5, 0.5, 0.5, 0.5]}
[INFO] [1732786455.887833533] [notebook_example]: Move to target 1


In [5]:
targets = [target0, target1]
traj = moveit2.plan_sequence(targets, blends=[0.1, 0.0])
# traj = moveit2.get_trajectory_from_sequence(result)

# if traj is not None:
#     moveit2.execute(traj)
#     success = moveit2.wait_until_executed()
if traj is not None:
    node.get_logger().info("Move to target 0")
    moveit2.execute(traj)
    success = moveit2.wait_until_executed()

[INFO] [1732789833.968677497] [notebook_example]: number of items in sequence: 2
[INFO] [1732789833.981623608] [notebook_example]: item 0 blend radius: 0.0
[INFO] [1732789833.983449693] [notebook_example]: item 0 request: moveit_msgs.msg.MotionPlanRequest(workspace_parameters=moveit_msgs.msg.WorkspaceParameters(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1732789833, nanosec=943327543), frame_id='base_link'), min_corner=geometry_msgs.msg.Vector3(x=-1.0, y=-1.0, z=-1.0), max_corner=geometry_msgs.msg.Vector3(x=1.0, y=1.0, z=1.0)), start_state=moveit_msgs.msg.RobotState(joint_state=sensor_msgs.msg.JointState(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1732789833, nanosec=935570384), frame_id=''), name=['shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint', 'shoulder_pan_joint'], position=[-1.5707963267948966, 1.5707963267948966, -7.632783294297951e-17, 0.0, 0.0, 0.0], velocity=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], effort

KeyboardInterrupt: 